In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

MergedFile = "merged_df.csv"
df = pd.read_csv(MergedFile, delimiter='\t')

# Add the columns directionality and chemotactic_index
directionality = []; chemotactic_index = []
IC_pos_x = -220;  IC_pos_y = 0
for index, row in df.iterrows():
    # Directionality: net displacement/total distance traveled
    if row['total_distance'] == 0: directionality.append(0)
    else: directionality.append( np.sqrt((row['position_x'] - IC_pos_x)**2 + (row['position_y'] - IC_pos_y)**2)/row['total_distance'] )
    # Chemotactic index: cos(theta) = (velocity*gradient)/(velocity_norm*gradient_norm)
    # Our case gradient/gradient_norm is (0,1) -> cos(theta) = v_y/velocity_norm
    velocity_norm = np.linalg.norm(np.array([row['velocity_x'],row['velocity_y']]))
    if velocity_norm == 0: chemotactic_index.append(0)
    else: chemotactic_index.append( row['velocity_y']/velocity_norm )
df['directionality'] = directionality 
df['chemotactic_index'] = chemotactic_index 

In [2]:
def PlotTraj_MeanStd(dataframe, var_name_y, axes, hue_var=None, hue_order=None, title=None, legend=False, PlotMeanStd=True):
    # Collapse x values by a tolerance
    tolerance_x = 5 # microns
    dataframe['position_x_bin'] = pd.cut(dataframe['position_x'], bins=np.arange(dataframe['position_x'].min(), dataframe['position_x'].max() + tolerance_x, tolerance_x), labels=False)
    if PlotMeanStd: 
        # Aggregate x values and substrate field by taking the mean and std
        if hue_var:
            df_agg_mean = dataframe.groupby(['position_x_bin', hue_var], as_index=False).agg({'position_x': 'mean', var_name_y: 'mean'})
            df_agg_std = dataframe.groupby(['position_x_bin', hue_var], as_index=False)[var_name_y].std()
        else:
            df_agg_mean = dataframe.groupby(['position_x_bin'], as_index=False).agg({'position_x': 'mean', var_name_y: 'mean'})
            df_agg_std = dataframe.groupby(['position_x_bin'], as_index=False)[var_name_y].std()
        # Plot mean curves
        plot1 = sns.lineplot(x="position_x", y=var_name_y, hue = hue_var,hue_order=["linear","exponential decay","half gaussian"], data=df_agg_mean, errorbar=None, ax=axes, legend=legend)
        if (var_name_y == "position_y"): plot1.set_ylim([-50,50])
        if title: plot1.set_title(title)
        if hue_var:
            if hue_order: hue_order_plt = hue_order
            else: hue_order_plt = df_agg_mean[hue_var].unique()
            # Plot std curves
            for subs_field in hue_order_plt:
                position_x = df_agg_mean.loc[df_agg_mean[hue_var] == subs_field]["position_x"].to_numpy()
                y_std_lb = df_agg_mean.loc[df_agg_mean[hue_var] == subs_field][var_name_y].to_numpy() - df_agg_std.loc[df_agg_std[hue_var] == subs_field][var_name_y].to_numpy()
                y_std_ub = df_agg_mean.loc[df_agg_mean[hue_var] == subs_field][var_name_y].to_numpy() + df_agg_std.loc[df_agg_std[hue_var] == subs_field][var_name_y].to_numpy()
                axes.fill_between(position_x, y1=y_std_lb, y2=y_std_ub, alpha=.25)
        else:
            position_x = df_agg_mean["position_x"].to_numpy()
            y_std_lb = df_agg_mean[var_name_y].to_numpy() - df_agg_std[var_name_y].to_numpy()
            y_std_ub = df_agg_mean[var_name_y].to_numpy() + df_agg_std[var_name_y].to_numpy()
            axes.fill_between(position_x, y1=y_std_lb, y2=y_std_ub, alpha=.25)
    else:
        plot1 = sns.lineplot(x="position_x", y=var_name_y, hue = hue_var,hue_order=["linear","exponential decay","half gaussian"], data=dataframe, estimator=None, ax=axes, legend=legend, units="replicateID", alpha=0.25)
    
    return plot1

# Metrics: *Speed*, *Directionality*, and *Chemotactic Index*

In [3]:
from ipywidgets import interactive
def PlotTrajs(sat_bias_value, sat_speed_value, hm_bias_value, hm_speed_value, PlotMeanStd=True):
    df_middle = df.loc[ (df['sat_bias'] == sat_bias_value) & (df['sat_speed'] == sat_speed_value) & (df['hm_bias'] == hm_bias_value) & (df['hm_speed'] == hm_speed_value) ]
    fig, ax = plt.subplots(figsize=(9,5))
    hue_order=["linear","exponential decay","half gaussian"]
    fig_title = f"The sample with sat_bias={sat_bias_value}, sat_speed={sat_speed_value}, hm_bias={hm_bias_value}, and hm_speed={hm_speed_value} ({len(df['replicateID'].unique())} replicates)"
    PlotTraj_MeanStd(df_middle, var_name_y='position_y', axes=ax, hue_var='substrate_field', hue_order=hue_order, title=fig_title, legend='full', PlotMeanStd=PlotMeanStd)

interactive_plot = interactive(PlotTrajs, sat_bias_value=df['sat_bias'].unique(), sat_speed_value=df['sat_speed'].unique(), hm_bias_value=df['hm_bias'].unique(), hm_speed_value=df['hm_speed'].unique())
display(interactive_plot)

interactive(children=(Dropdown(description='sat_bias_value', options=(1.0, 0.75, 0.5), value=1.0), Dropdown(de…

In [5]:
def PlotQoIs(sat_bias_value, sat_speed_value, hm_bias_value, hm_speed_value, QOI, PlotMeanStd=True):
    df_middle = df.loc[ (df['sat_bias'] == sat_bias_value) & (df['sat_speed'] == sat_speed_value) & (df['hm_bias'] == hm_bias_value) & (df['hm_speed'] == hm_speed_value) ]
    fig, ax = plt.subplots(figsize=(9,5))
    hue_order=["linear","exponential decay","half gaussian"]
    fig_title = f"The {QOI} with sat_bias={sat_bias_value}, sat_speed={sat_speed_value}, hm_bias={hm_bias_value}, and hm_speed={hm_speed_value} ({len(df['replicateID'].unique())} replicates)"
    PlotTraj_MeanStd(df_middle, var_name_y=QOI, axes=ax, hue_var='substrate_field', hue_order=hue_order, title=fig_title, legend='full', PlotMeanStd=PlotMeanStd)
interactive_plot = interactive(PlotQoIs, sat_bias_value=df['sat_bias'].unique(), sat_speed_value=df['sat_speed'].unique(), hm_bias_value=df['hm_bias'].unique(), hm_speed_value=df['hm_speed'].unique(), QOI=["average_speed","directionality","chemotactic_index"])
display(interactive_plot)

interactive(children=(Dropdown(description='sat_bias_value', options=(1.0, 0.75, 0.5), value=1.0), Dropdown(de…

## **Speed:** The average velocity at which a cell moves. This is often calculated as the total distance traveled divided by the position_x taken.
$$Speed = \frac{Total~distance~traveled}{Time~taken}$$

In [ ]:
fig, ax = plt.subplots(figsize=(9,5))
hue_order=["linear","exponential decay","half gaussian"]
fig_title = f"The mean and std of speed from {len(df['sampleID'].unique())} samples  and {len(df['replicateID'].unique())} replicates"
PlotTraj_MeanStd(df, var_name_y='average_speed', axes=ax, hue_var='substrate_field', hue_order=hue_order, title=fig_title, legend='full')

In [ ]:
# Fix parameter: sat_bias
hue_order=["linear","exponential decay","half gaussian"]
col_order=[0.5,0.75,1.0]
g1 = sns.FacetGrid(df, hue="substrate_field", hue_order=hue_order, col="sat_bias", col_order=col_order)
for ax, col_value in zip(g1.axes[0], col_order):
    if col_value == col_order[1]:
        plot_center = PlotTraj_MeanStd(df.loc[df['sat_bias'] == col_value], var_name_y='average_speed', hue_var="substrate_field", hue_order=hue_order, axes=ax, legend="full")
    else:
        plot_side = PlotTraj_MeanStd(df.loc[df['sat_bias'] == col_value], var_name_y='average_speed', hue_var="substrate_field", hue_order=hue_order, axes=ax)
g1.set_xlabels("")
sns.move_legend(plot_center, "lower center", bbox_to_anchor=(0.5, 1.1), ncol=3, frameon=True)#, title=None)

# Fix parameter: sat_speed
col_order=[0.5,0.75,1.0]
g2 = sns.FacetGrid(df, hue="substrate_field", hue_order=hue_order, col="sat_speed", col_order=col_order)
for ax, col_value in zip(g2.axes[0], col_order):
    if col_value == col_order[1]:
        plot_center = PlotTraj_MeanStd(df.loc[df['sat_speed'] == col_value], var_name_y='average_speed', hue_var="substrate_field", hue_order=hue_order, axes=ax, legend="full")
    else:
        plot_side = PlotTraj_MeanStd(df.loc[df['sat_speed'] == col_value], var_name_y='average_speed', hue_var="substrate_field", hue_order=hue_order, axes=ax)
g2.set_xlabels("")

# Fix parameter: hm_bias
col_order=[0.001,0.003,0.005]
g3 = sns.FacetGrid(df, hue="substrate_field", hue_order=hue_order, col="hm_bias", col_order=col_order)
for ax, col_value in zip(g3.axes[0], col_order):
    if col_value == col_order[1]:
        plot_center = PlotTraj_MeanStd(df.loc[df['hm_bias'] == col_value], var_name_y='average_speed', hue_var="substrate_field", hue_order=hue_order, axes=ax, legend="full")
    else:
        plot_side = PlotTraj_MeanStd(df.loc[df['hm_bias'] == col_value], var_name_y='average_speed', hue_var="substrate_field", hue_order=hue_order, axes=ax)
g3.set_xlabels("")

# Fix parameter: hm_speed
col_order=[0.001,0.003,0.005]
g4 = sns.FacetGrid(df, hue="substrate_field", hue_order=hue_order, col="hm_speed", col_order=col_order)
for ax, col_value in zip(g4.axes[0], col_order):
    if col_value == col_order[1]:
        plot_center = PlotTraj_MeanStd(df.loc[df['hm_speed'] == col_value], var_name_y='average_speed', hue_var="substrate_field", hue_order=hue_order, axes=ax, legend="full")
    else:
        plot_side = PlotTraj_MeanStd(df.loc[df['hm_speed'] == col_value], var_name_y='average_speed', hue_var="substrate_field", hue_order=hue_order, axes=ax)

## **Directionality:** A measure of how straight the cell’s path is. It is often quantified as the ratio of the net displacement (the straight-line distance from start to end point) to the total path length traveled by the cell.
$$Directionality = \frac{Straight-line distance}{Total~distance~traveled}$$

In [ ]:
fig, ax = plt.subplots(figsize=(9,5))
fig_title = f"The mean and std of directionality from {len(df['sampleID'].unique())} samples  and {len(df['replicateID'].unique())} replicates"
PlotTraj_MeanStd(df, var_name_y='directionality', axes=ax, hue_var='substrate_field', hue_order=hue_order, title=fig_title, legend='full')

In [ ]:
# Fix parameter: sat_bias
hue_order=["linear","exponential decay","half gaussian"]
col_order=[0.5,0.75,1.0]
g1 = sns.FacetGrid(df, hue="substrate_field", hue_order=hue_order, col="sat_bias", col_order=col_order)
for ax, col_value in zip(g1.axes[0], col_order):
    if col_value == col_order[1]:
        plot_center = PlotTraj_MeanStd(df.loc[df['sat_bias'] == col_value], var_name_y='directionality', hue_var="substrate_field", hue_order=hue_order, axes=ax, legend="full")
    else:
        plot_side = PlotTraj_MeanStd(df.loc[df['sat_bias'] == col_value], var_name_y='directionality', hue_var="substrate_field", hue_order=hue_order, axes=ax)
g1.set_xlabels("")
sns.move_legend(plot_center, "lower center", bbox_to_anchor=(0.5, 1.1), ncol=3, frameon=True)#, title=None)

# Fix parameter: sat_speed
col_order=[0.5,0.75,1.0]
g2 = sns.FacetGrid(df, hue="substrate_field", hue_order=hue_order, col="sat_speed", col_order=col_order)
for ax, col_value in zip(g2.axes[0], col_order):
    if col_value == col_order[1]:
        plot_center = PlotTraj_MeanStd(df.loc[df['sat_speed'] == col_value], var_name_y='directionality', hue_var="substrate_field", hue_order=hue_order, axes=ax, legend="full")
    else:
        plot_side = PlotTraj_MeanStd(df.loc[df['sat_speed'] == col_value], var_name_y='directionality', hue_var="substrate_field", hue_order=hue_order, axes=ax)
g2.set_xlabels("")

# Fix parameter: hm_bias
col_order=[0.001,0.003,0.005]
g3 = sns.FacetGrid(df, hue="substrate_field", hue_order=hue_order, col="hm_bias", col_order=col_order)
for ax, col_value in zip(g3.axes[0], col_order):
    if col_value == col_order[1]:
        plot_center = PlotTraj_MeanStd(df.loc[df['hm_bias'] == col_value], var_name_y='directionality', hue_var="substrate_field", hue_order=hue_order, axes=ax, legend="full")
    else:
        plot_side = PlotTraj_MeanStd(df.loc[df['hm_bias'] == col_value], var_name_y='directionality', hue_var="substrate_field", hue_order=hue_order, axes=ax)
g3.set_xlabels("")

# Fix parameter: hm_speed
col_order=[0.001,0.003,0.005]
g4 = sns.FacetGrid(df, hue="substrate_field", hue_order=hue_order, col="hm_speed", col_order=col_order)
for ax, col_value in zip(g4.axes[0], col_order):
    if col_value == col_order[1]:
        plot_center = PlotTraj_MeanStd(df.loc[df['hm_speed'] == col_value], var_name_y='directionality', hue_var="substrate_field", hue_order=hue_order, axes=ax, legend="full")
    else:
        plot_side = PlotTraj_MeanStd(df.loc[df['hm_speed'] == col_value], var_name_y='directionality', hue_var="substrate_field", hue_order=hue_order, axes=ax)

## **Chemotactic Index (CI):** This index measures the efficiency of directed movement towards or away from the chemical source. It is calculated as the cosine of the angle between the cell's movement vector and the gradient vector. A CI close to 1 indicates strong chemotaxis toward the source, while a CI close to -1 indicates movement away from the source.
$$CI = \cos{\theta} = \frac{\vec{v}\cdot\vec{g}}{|\vec{v}||\vec{g}|}$$

In [ ]:
fig, ax = plt.subplots(figsize=(9,5))
fig_title = f"The mean and std of chemotactic_index from {len(df['sampleID'].unique())} samples  and {len(df['replicateID'].unique())} replicates"
PlotTraj_MeanStd(df, var_name_y='chemotactic_index', axes=ax, hue_var='substrate_field', hue_order=hue_order, title=fig_title, legend='full')

In [ ]:
# Fix parameter: sat_bias
hue_order=["linear","exponential decay","half gaussian"]
col_order=[0.5,0.75,1.0]
g1 = sns.FacetGrid(df, hue="substrate_field", hue_order=hue_order, col="sat_bias", col_order=col_order)
for ax, col_value in zip(g1.axes[0], col_order):
    if col_value == col_order[1]:
        plot_center = PlotTraj_MeanStd(df.loc[df['sat_bias'] == col_value], var_name_y='chemotactic_index', hue_var="substrate_field", hue_order=hue_order, axes=ax, legend="full")
    else:
        plot_side = PlotTraj_MeanStd(df.loc[df['sat_bias'] == col_value], var_name_y='chemotactic_index', hue_var="substrate_field", hue_order=hue_order, axes=ax)
g1.set_xlabels("")
sns.move_legend(plot_center, "lower center", bbox_to_anchor=(0.5, 1.1), ncol=3, frameon=True)#, title=None)

# Fix parameter: sat_speed
col_order=[0.5,0.75,1.0]
g2 = sns.FacetGrid(df, hue="substrate_field", hue_order=hue_order, col="sat_speed", col_order=col_order)
for ax, col_value in zip(g2.axes[0], col_order):
    if col_value == col_order[1]:
        plot_center = PlotTraj_MeanStd(df.loc[df['sat_speed'] == col_value], var_name_y='chemotactic_index', hue_var="substrate_field", hue_order=hue_order, axes=ax, legend="full")
    else:
        plot_side = PlotTraj_MeanStd(df.loc[df['sat_speed'] == col_value], var_name_y='chemotactic_index', hue_var="substrate_field", hue_order=hue_order, axes=ax)
g2.set_xlabels("")

# Fix parameter: hm_bias
col_order=[0.001,0.003,0.005]
g3 = sns.FacetGrid(df, hue="substrate_field", hue_order=hue_order, col="hm_bias", col_order=col_order)
for ax, col_value in zip(g3.axes[0], col_order):
    if col_value == col_order[1]:
        plot_center = PlotTraj_MeanStd(df.loc[df['hm_bias'] == col_value], var_name_y='chemotactic_index', hue_var="substrate_field", hue_order=hue_order, axes=ax, legend="full")
    else:
        plot_side = PlotTraj_MeanStd(df.loc[df['hm_bias'] == col_value], var_name_y='chemotactic_index', hue_var="substrate_field", hue_order=hue_order, axes=ax)
g3.set_xlabels("")

# Fix parameter: hm_speed
col_order=[0.001,0.003,0.005]
g4 = sns.FacetGrid(df, hue="substrate_field", hue_order=hue_order, col="hm_speed", col_order=col_order)
for ax, col_value in zip(g4.axes[0], col_order):
    if col_value == col_order[1]:
        plot_center = PlotTraj_MeanStd(df.loc[df['hm_speed'] == col_value], var_name_y='chemotactic_index', hue_var="substrate_field", hue_order=hue_order, axes=ax, legend="full")
    else:
        plot_side = PlotTraj_MeanStd(df.loc[df['hm_speed'] == col_value], var_name_y='chemotactic_index', hue_var="substrate_field", hue_order=hue_order, axes=ax)

## RBD-FAST - Random Balance Designs Fourier Amplitude Sensitivity Test

**Random Balance Design (RBD):** This part ensures efficient sampling of the input parameter space. RBD creates a set of model runs where each input parameter value appears in combination with all other possible values from the other input parameters. This avoids bias in the analysis.

**Fourier Amplitude Sensitivity Test (FAST):** This method uses the generated model runs from RBD to assess the influence of each input parameter on the model output. It analyzes the variance in the output caused by changes in each input parameter. This helps identify which input parameters have the most significant impact on the final outcome.

**Benefits of RBD-FAST:**

**Efficiency:** It requires fewer model runs compared to other SA methods for achieving similar accuracy.
**Global Sensitivity Indices:** It provides not just qualitative information (like which parameters are important) but also quantitative measures (sensitivity indices) that tell you how much each parameter contributes to the output uncertainty.

**Applications of RBD-FAST:**
**Uncertainty Quantification:** Used in various fields to understand how uncertainties in model inputs propagate to uncertainties in the model outputs. This is helpful in areas like engineering, physics, and environmental modeling.
**Model Calibration and Validation:** Helps identify key parameters that need to be accurately determined for reliable model predictions.

In [ ]:
from SALib.analyze.rbd_fast import analyze as analyze_rbd_fast

params_list = ['substrate_field', 'sat_bias', 'sat_speed', 'hm_bias', 'hm_speed']
problem = {'num_vars': len(params_list), 'names': params_list}

# select the last point of each trajectory
idx = df.groupby(['sampleID','replicateID'])['position_x'].idxmax()
parameters = df.groupby(['sampleID'])[params_list].max().to_numpy()
speed_lastPoint = df.loc[idx].groupby(['sampleID'])['average_speed'].mean().to_numpy() # average of all replicates at last point
direc_lastPoint = df.loc[idx].groupby(['sampleID'])['directionality'].mean().to_numpy() # average of all replicates at last point
CI_mean = df.groupby(['sampleID'])['chemotactic_index'].mean().to_numpy() # average of all replicates in the entire trajectories

print(pars.shape,speed_lastPoint.shape,direc_lastPoint.shape,CI_mean.shape)

In [ ]:
Si = analyze_rbd_fast(problem, parameters, speed_lastPoint, print_to_console=True)
Si.plot()

In [ ]:
Si = analyze_rbd_fast(problem, parameters, direc_lastPoint, print_to_console=True)
Si.plot()

In [ ]:
Si = analyze_rbd_fast(problem, parameters, CI_mean, print_to_console=True)
Si.plot()